In [297]:
import random
#global variables
suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
values = {'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5, 'Six': 6, 'Seven': 7, 'Eight': 8, 'Nine': 9, 
          'Ten': 10, 'Jack': 10, 'Queen': 10, 'King': 10, 'Ace': 11}

In [298]:
class Deck:
    #creates a deck, list of tuples (rank, suit)
    def __init__(self):
        self.deck = []
        for suit in suits:
            for rank in ranks:
                self.deck.append(Card(suit, rank))
    
    def __str__(self):
        deck_string = ''
        for card in self.deck:
            deck_string += f'{card.get_rank()} of {card.get_suit()}\n'
        return deck_string
        
    #shuffles deck
    def shuffle(self):
        random.shuffle(self.deck)
        
    #removes then returns removed card
    def remove_card(self):
        return self.deck.pop(0)
        

In [299]:
class Card:
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank
        self.value = values[self.rank]
        
    def __str__(self):
        return f'{self.rank} of {self.suit}'
    
    def get_card_value(self):
        return self.value
    
    def get_suit(self):
        return self.suit
    
    def get_rank(self):
        return self.rank

In [300]:
class Hand:
    def __init__(self):
        self.player_hand = []
        
    def __str__(self):
        hand_string = ''
        for card in self.player_hand:
            hand_string += f'{card.get_rank()} of {card.get_suit()}\n'
        return hand_string
    
    def get_card(self, index):
        return self.player_hand[index]
    
    def add_card(self, card):
        self.player_hand.append(card)
        
    def get_hand_count(self):
        count = 0
        num_aces = 0
        for card in self.player_hand:
            count += card.get_card_value()
            if card.get_rank() in ['Ace']: num_aces += 1
        # if count is greater than 21, ace adjusts to be value of 1
        while count > 21 and num_aces > 0:
            count -= 10
            num_aces -= 1
        return count

In [301]:
class Chip:
    def __init__(self):
        self.total_chips = 1000
        self.bet = -1
    
    def set_bet(self, amount):
        self.bet = amount
        
    def get_chips(self):
        return self.total_chips
    
    def get_bet(self):
        return self.bet
    
    def won_bet(self):
        self.total_chips += self.bet
        
    def lost_bet(self):
        self.total_chips -= self.bet
        

In [317]:
def place_bet(chip):
    # check if the value entered is an int
    while True:
        try:
            in_bet = int(input(f"\nPlace your bet (Your total chips: ${chip.get_chips()}): "))
            break
        except ValueError:
            print("Not a number, please try again!")
            
    # we established in_bet is an int, now checking is it's a valid int
    while True:
        if(in_bet > chip.total_chips):
            print("Insufficient funds, please try again!")
        elif(in_bet <= 0):
            print("Bet must be greater than $0, try again!")
        else:
            break
        in_bet = int(input(f"Place your bet (Your total chips: ${chip.get_chips()}): "))
    
    # place bet
    chip.set_bet(in_bet)

In [303]:
def deal(player, cpu, deck):
    for i in range(0,2):
        player.add_card(deck.remove_card())
        cpu.add_card(deck.remove_card())

In [319]:
def hit_or_stand(player, deck):
    # make sure user inputs 'hit' or 'stand'
    while True:
        choice = input(f'\nWould you like to hit or stand? (Your count is {player.get_hand_count()}): ')
        if choice not in ['hit', 'Hit', 'stand', 'Stand']:
            print("Please enter 'hit' or 'stand'.")
        else: break
            
    #stand ends turn
    if choice in ['stand', 'Stand']:
        return 'Stand'
    #hit would entail busting or not busting
    else:
        player.add_card(deck.remove_card())
        if player.get_hand_count() == 21:
            return 'Black Jack'
        elif player.get_hand_count() > 21:
            return 'Bust'
            

In [311]:
# if turn is True, second card for dealer must not be revealed
# if turn is False, all dealer's cards must be revealed
def print_hands(player, cpu, player_turn):
    print('__________________________________')
    if player_turn:
        print(f"\nDealer's Hand: (Count = {cpu.get_hand_count()})\n")
        print(cpu)
    else:
        print(f"\nDealer's Hand:\n{cpu.get_card(0)}\n???\n")
    print(f"Player's Hand: (Count = {player.get_hand_count()})")
    print(player)
    print('__________________________________')

In [306]:
def play_again_prompt():
    repeat = input("Want to keep playing? (y/n): ")
    while repeat not in ['Y', 'N', 'n', 'y']:
        repeat = input("Please input 'y' or 'n': ")
    if repeat in ['Y', 'y']: return True
    else: return False

In [315]:
def welcome_screen():
    print("Welcome to Black Jack!\n")
    print("You and the Dealer will compete to see who can get the closest to a count of 21, good luck!\n")
    print("_________________________________________________________________________________")

In [329]:
play_again = True
player_chips = Chip()
welcome_screen()
while play_again and player_chips.get_chips() > 0:
    player_hand = Hand()
    cpu_hand = Hand()
    deck = Deck()
    deck.shuffle()
    player_done = False
    player_blackjack = True
    #place bet
    place_bet(player_chips)

    #deal cards
    deal(player_hand, cpu_hand, deck)

    #reveal one cpu card, both player cards
    print_hands(player_hand, cpu_hand, False)

    #if player hand is 21, no hit or stand option
    if player_hand.get_hand_count() == 21:
        print("Black Jack!")
    else:
        while True:
            string = hit_or_stand(player_hand, deck)
            if string in ['Bust']:
                print_hands(player_hand, cpu_hand, False)
                print("Bust! You lose!")
                player_chips.lost_bet()
                player_done = True
                break
            elif string in ['Black Jack']:
                print_hands(player_hand, cpu_hand, False)
                print('Black Jack! You win!')
                player_chips.won_bet()
                player_done = True
                break
            elif string in ['Stand']:
                print_hands(player_hand, cpu_hand, False)
                break
            print_hands(player_hand, cpu_hand, False)
           
    if player_chips.get_chips() == 0:
        print("\nOut of chips!")
        break
    elif player_done: 
        play_again = play_again_prompt()
        continue
        
    #cpu's turn, if player doesnt bust, dealer keeps hitting until their count reaches 17 or more
    print_hands(player_hand, cpu_hand, True)
    while cpu_hand.get_hand_count() < 17:
        cpu_hand.add_card(deck.remove_card())
        print_hands(player_hand, cpu_hand, True)
    
    if cpu_hand.get_hand_count() == 21:
        print("\nBlack Jack! You Lose!")
        player_chips.lost_bet()
    elif cpu_hand.get_hand_count() > 21:
        print("\nBust! You win!")
        player_chips.won_bet()
    elif cpu_hand.get_hand_count() > player_hand.get_hand_count():
        print("\nYou lose!")
        player_chips.lost_bet()
    elif cpu_hand.get_hand_count() < player_hand.get_hand_count():
        print("\nYou win!")
        player_chips.won_bet()
    else:
        print("\nPush!")
    
    if player_chips.get_chips() == 0:
        print("\nOut of chips!")
    else: play_again = play_again_prompt()
print("\nThanks for playing!")

Welcome to Black Jack!

You and the Dealer will compete to see who can get the closest to a count of 21, good luck!

_________________________________________________________________________________

Place your bet (Your total chips: $1000): 1000
__________________________________

Dealer's Hand:
Queen of Spades
???

Player's Hand: (Count = 21)
Queen of Hearts
Ace of Clubs

__________________________________
Black Jack!
__________________________________

Dealer's Hand: (Count = 19)

Queen of Spades
Nine of Hearts

Player's Hand: (Count = 21)
Queen of Hearts
Ace of Clubs

__________________________________

You win!
Want to keep playing? (y/n): y

Place your bet (Your total chips: $2000): 2000
__________________________________

Dealer's Hand:
Seven of Hearts
???

Player's Hand: (Count = 6)
Four of Spades
Two of Diamonds

__________________________________

Would you like to hit or stand? (Your count is 6): hit
__________________________________

Dealer's Hand:
Seven of Hearts
???

Pla